# PS2-6: Spam Classification

## Import Library

In [266]:
import collections
from os.path import split

import numpy as np

import Full_Problem_Set.PS2.src.svm as svm
import Full_Problem_Set.PS2.src.util as util

## Load Dataset

In [267]:
#Train
x_train, y_train = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_train.tsv')

#Valid
x_valid, y_valid = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_val.tsv')

#Test
x_test, y_test = util.load_spam_dataset('/home/anhnt02/Desktop/CS229-Fall2018-FullCourse/Full_Problem_Set/PS2/data/ds6_test.tsv')

## a) Process Spam messages into Numpy array

### 1. Normalization & Splitting

In [268]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    #1. Normalization
    lower_message = message.lower()

    #2. Splitting
    split_message_to_words = lower_message.split()

    return split_message_to_words
    # *** END CODE HERE ***

### 2. Create a dictionary mapping words to integer indices

In [269]:
def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    #1. Convert dataset into array
    messages_arr = np.array(messages)

    #2. Create new dict
    dict_word_indices = {}

    #3. Convert each message in dataset to array
    words_msg_arr = np.empty(len(messages_arr), dtype=object)
    for i in range(len(messages_arr)):
        words_msg_arr[i] = get_words(messages_arr[i])

    #4. Separate words_msg_arr into each word and count the number of it appears
    for i in range(len(words_msg_arr)):
        for j in range(len(words_msg_arr[i])):
            word = words_msg_arr[i][j]
            if word not in dict_word_indices:
                dict_word_indices[word] = 1
            else:
                dict_word_indices[word] += 1

    #5. Delete word in dict that has less than 5 times appear
    for key, value in list(dict_word_indices.items()):
        if value < 5:
            del dict_word_indices[key]

    #6. Convert "the number of word appear" into index (This step is important for retrieve process below)
    cnt = -1
    for key, value in dict_word_indices.items():
        cnt += 1
        dict_word_indices[key] = cnt

    return dict_word_indices
    # *** END CODE HERE ***

### 3. Generate Feature Matrix

In [270]:
def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    #1. Convert each message into word
    word_msg_arr = [None] * len(messages)
    for i in range(len(messages)):
        word_msg_arr[i] = get_words(messages[i])

    #.2 Create matrix feature (row - message[i]; column - dict of word; each entry is the times of word(in dict) appear in each message[i])
    feature_matrix = np.zeros((len(messages), len(word_dictionary)))

    for i in range(len(word_msg_arr)):
        for j in range(len(word_msg_arr[i])):
            if word_msg_arr[i][j] in word_dictionary:
                key = word_msg_arr[i][j]
                value = word_dictionary[key]
                feature_matrix[i][value] += 1

    return feature_matrix
    # *** END CODE HERE ***

## b) Implement Naive Bayes classifier for spam classification with Multinomial Event Model

### 1. Fit Naive Bayes Model

In [271]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    #1. Get feature_matrix shape
    m, n = matrix.shape

    #-----------------------------------------------------
    #2. Calculate Phi_y
    phi_y = 0
    for i in range(m):
        if(labels[i] == 1):
            phi_y += 1 /m * 1
    #-----------------------------------------------------
    #3. Calculate Phi_k_y=1
    #numerator: Count number of k appear in class y = 1
    k_appear_cnt_y1 = np.zeros(n)
    for i in range(m):
        if labels[i] == 1:
            k_appear_cnt_y1 += matrix[i]

    #denominator: Count number of word in class y = 1
    #METHOD 1: COUNT ALL THE NUMBER OF EACH CLASS Y = 1
    # total_word_y1 = 0
    # for i in range(m):
    #     if labels[i] == 1:
    #         for j in range(n):
    #             total_word_y1 += matrix[i][j]

    #METHOD 2: CLASS 1 JUST CONTAIN ALL WORD IN k_appear_cnt_y1
    total_word_y1 = 0
    for j in range(n):
        total_word_y1 += k_appear_cnt_y1[j]

    #Apply Laplace Smoothing
    phi_k_y1 = (1 + k_appear_cnt_y1) / (total_word_y1 + n)
    #-----------------------------------------------------
    #4. Calculate Phi_k_y=0
    #numerator: Count number of k appear in class y = 0
    k_appear_cnt_y2 = np.zeros(n)
    for i in range(m):
        if labels[i] == 0:
            k_appear_cnt_y2 += matrix[i]

    #denominator: Count number of word in class y = 0
    total_word_y2 = 0
    for j in range(n):
        total_word_y2 += k_appear_cnt_y2[j]

    #Apply Laplace Smoothing
    phi_k_y2 = (1 + k_appear_cnt_y2) / (total_word_y2 + n)
    #-----------------------------------------------------

    return phi_y, phi_k_y1, phi_k_y2
    # *** END CODE HERE ***